In [23]:
import pandas as pd

import numpy as np


In [24]:
expId = 126
caseStudyId = 12
predDf = pd.read_csv(
    f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_{caseStudyId}/case_study_pred.csv"
)
gtDf = pd.read_csv(
    f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_{caseStudyId}/case_study_gt.csv"
)
visionPredDf1 = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expId}/Saloon - 4 Dr/Saloon - 4 Dr_imgs_pred_output.csv")
visionPredDf2 = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expId}/Hatchback - 5 Dr/Hatchback - 5 Dr_imgs_pred_output.csv")
visionPredDf3 = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/{expId}/SUV - 5 Dr/SUV - 5 Dr_imgs_pred_output.csv")
visionPredDf = pd.concat([visionPredDf1, visionPredDf2,visionPredDf3 ])
majorParts = [x.replace("vision_", "") for x in  visionPredDf.columns]
majorParts

['Unnamed: 0',
 'CaseID',
 'bonnet',
 'bumper_front',
 'engine',
 'fender_front_rh',
 'front_panel',
 'grille',
 'headlamp_rh',
 'non_external',
 'windscreen_front',
 'bumper_rear',
 'rear_compartment',
 'rear_panel',
 'rear_quarter_rh',
 'tail_lamp_rh',
 'windscreen_rear',
 'rear_quarter_lh',
 'tail_lamp_lh',
 'fender_front_lh',
 'headlamp_lh']

In [25]:
len(predDf)

30

In [26]:
targetCols = [x for x in predDf.columns if "0" not in x and x != "CaseID"]
allRows = []
for c in targetCols:
    for caseId in predDf["CaseID"].unique().tolist():
        visionPred = -1
        if(c in majorParts):
            visionPred = visionPredDf[visionPredDf["CaseID"] == caseId][f"vision_{c}"].item()
            

        pred = predDf[predDf["CaseID"] == caseId][c].item()
        gt = gtDf[gtDf["CaseID"] == caseId][c].item()
        fp = 1 if gt == 0 and pred == 1 else 0
        fn = 1 if gt == 1 and pred == 0 else 0
        allRows.append(
            {
                "CaseID" : caseId,
                "part" : c,
                "v_pred":visionPred,
                "pred" : pred,
                "gt" : gt,
                "fp" : fp, 
                "fn" : fn,
                "inspect" : None,
                "remarks" : None

            }
        )
caseStudyInspectDf = pd.json_normalize(allRows)
caseStudyInspectDf.sort_values(by = ["CaseID", "part"], inplace=True)

In [27]:
caseStudyInspectDf

,CaseID,part,v_pred,pred,gt,fp,fn,inspect,remarks
60,10047111,air_conditioning,-1,0,0,0,0,None,None
120,10047111,airbag,-1,0,0,0,0,None,None
240,10047111,bonnet,0,0,0,0,0,None,None
270,10047111,bumper_front,0,0,0,0,0,None,None
390,10047111,bumper_rear,0,0,0,0,0,None,None
...,...,...,...,...,...,...,...,...,...
599,13025161,tail_lamp_lh,0,0,0,0,0,None,None
629,13025161,tail_lamp_rh,0,0,0,0,0,None,None
119,13025161,undercarriage,-1,1,1,0,0,None,None
659,13025161,windscreen_front,0,0,0,0,0,None,None


In [28]:
len(caseStudyInspectDf["part"].unique())

26

In [29]:
majorParts = 25 * 30
majorParts

750

In [30]:
154 / majorParts

0.20533333333333334

In [31]:
caseStudyWrong = caseStudyInspectDf[(caseStudyInspectDf["pred"] !=caseStudyInspectDf["gt"] )
                                    & (caseStudyInspectDf["v_pred"] != -1)
                                    
                                    ]
caseStudyWrong

,CaseID,part,v_pred,pred,gt,fp,fn,inspect,remarks
480,10047111,fender_front_lh,1,1,0,1,0,None,None
421,10049081,engine,0,0,1,0,1,None,None
451,10049081,front_panel,1,1,0,1,0,None,None
691,10049081,rear_panel,1,1,0,1,0,None,None
721,10049081,rear_quarter_rh,1,1,0,1,0,None,None
...,...,...,...,...,...,...,...,...,...
419,13025161,bumper_rear,0,0,1,0,1,None,None
449,13025161,engine,0,0,1,0,1,None,None
479,13025161,front_panel,1,1,0,1,0,None,None
329,13025161,grille,0,0,1,0,1,None,None


In [32]:
len(caseStudyWrong) / len(caseStudyInspectDf)

0.09743589743589744

In [33]:

caseStudyWrongVis = caseStudyInspectDf[(caseStudyInspectDf["pred"] !=caseStudyInspectDf["v_pred"]) & (caseStudyInspectDf["v_pred"] != -1) ]
caseStudyWrongVis

,CaseID,part,v_pred,pred,gt,fp,fn,inspect,remarks


In [34]:
# majorPartDf = caseStudyInspectDf[caseStudyInspectDf["v_pred"] != - 1]
majorPartDf = caseStudyInspectDf

In [35]:
majorPartDf["part"].unique()

array(['air_conditioning', 'airbag', 'bonnet', 'bumper_front',
       'bumper_rear', 'cooling_fan', 'engine', 'fender_front_lh',
       'fender_front_rh', 'fog_lamp_lh', 'fog_lamp_rh', 'front_panel',
       'grille', 'headlamp_lh', 'headlamp_rh', 'interior', 'radiator',
       'rear_compartment', 'rear_panel', 'rear_quarter_lh',
       'rear_quarter_rh', 'tail_lamp_lh', 'tail_lamp_rh', 'undercarriage',
       'windscreen_front', 'windscreen_rear'], dtype=object)

In [36]:
fpr = len(majorPartDf[(majorPartDf["pred"]  == 1) & (majorPartDf["gt"]  == 0) ]) / len(majorPartDf[ (majorPartDf["gt"]  == 0)])
fpr

0.10832025117739404

In [37]:
fnr = len(majorPartDf[(majorPartDf["pred"]  == 0) & (majorPartDf["gt"]  == 1) ]) / len(majorPartDf[ (majorPartDf["gt"]  == 1)])
fnr

0.2727272727272727

In [38]:
acc = len(majorPartDf[(majorPartDf["pred"]  == majorPartDf["gt"])]) / len(majorPartDf)
acc

0.8615384615384616

In [39]:
allSubsetAcc = []
for caseId in majorPartDf["CaseID"].unique().tolist():
    rows = majorPartDf[majorPartDf["CaseID"] == caseId]
    correct = rows[rows["pred"] == rows["gt"]]
    subset_acc = len(correct) / len(rows)
    allSubsetAcc.append(subset_acc)
    
    # print(subset_acc)
print(np.mean(allSubsetAcc))

0.8615384615384616


In [40]:
caseStudyInspectDf.to_csv("./case_study_result.csv")